In [30]:
# | default_exp core.agent

In [6]:
# | export
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
import json
from agentic.llms.client import LLMClient
from agentic.configs.loader import get_model_config
from agentic.tools.manager import ToolManager
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [7]:
# | export
@dataclass
class Message:
    role: str
    content: str
    tool_calls: Optional[List[Dict]] = None
    tool_call_id: Optional[str] = None

@dataclass
class AgentConfig:
    name: str
    instructions: str = ""
    model: Optional[str] = None
    tools: List[str] = field(default_factory=list)
    temperature: float = 0.7
    max_tokens: Optional[int] = None

class Agent:
    """Core Agent class with tool execution and conversation management"""

    def __init__(self, config: AgentConfig, llm_client: Optional[LLMClient] = None):
        self.config = config
        self.system_prompt = config.instructions
        self.llm_client = llm_client or self._create_default_llm_client()
        logger.info(f"Initialized LLM client with model: {self.llm_client.model}")
        self.conversation_history: List[Message] = [Message(role="system", content=self.system_prompt)]
        self.tools_registry: Dict[str, Callable] = {}
        self.guardrails: List[Callable] = []
        self.tool_manager = ToolManager()

    def _create_default_llm_client(self) -> LLMClient:
        """Create default LLM client from config."""
        model_config = get_model_config()
        model_name = self.config.model or model_config.get('name')
        if not model_name:
            raise ValueError("No model specified in config")
        return LLMClient(
            model=model_name,
            base_url=model_config.get('url'),
            api_key=model_config.get('api_key')
        )

    def add_tool(self, name: str, tool_func: Callable) -> None:
        """Register a tool with the agent."""
        if not callable(tool_func):
            raise ValueError(f"Tool function '{name}' must be callable")
        self.tools_registry[name] = tool_func

    def add_guardrail(self, guardrail_func: Callable) -> None:
        """Add a guardrail function."""
        self.guardrails.append(guardrail_func)

    def run(self, message: str, **kwargs) -> Dict[str, Any]:
        """Execute agent with message and return response."""
        # Apply guardrails
        for guardrail in self.guardrails:
            result = guardrail(message)
            if not isinstance(result, bool) or not result:
                return {"content": "Request blocked by guardrails", "blocked": True}

        # Add user message
        self.conversation_history.append(Message(role="user", content=message))

        # Initialize result
        final_result = {"content": "", "tool_calls": [], "blocked": False}
        max_iterations = kwargs.get('max_iterations', 10)  # Fallback safety

        while True:
            # Get available tools
            available_tools = self._get_available_tools()

            # Create completion
            messages = self._format_messages_for_llm()
            stream = kwargs.get('stream', True)
            try:
                response = self.llm_client.create_completion(
                    messages=messages,
                    tools=available_tools,
                    **kwargs
                )
            except Exception as e:
                logger.error(f"LLM completion failed: {str(e)}")
                return {"content": f"Error: {str(e)}", "blocked": True}

            # Process response
            if stream:
                if not hasattr(response, '__iter__'):
                    raise ValueError("Streaming response expected but non-iterable response received")
                try:
                    result = self.llm_client.handle_streaming_response(response)
                except Exception as e:
                    logger.error(f"Error processing streaming response: {str(e)}")
                    return {"content": f"Streaming error: {str(e)}", "blocked": True}
            else:
                try:
                    result = self.llm_client.process_response(response)
                except Exception as e:
                    logger.error(f"Error processing response: {str(e)}")
                    return {"content": f"Response error: {str(e)}", "blocked": True}

            # Add assistant response to history
            assistant_message = Message(
                role="assistant",
                content=result.get("content", ""),
                tool_calls=result.get("tool_calls")
            )
            self.conversation_history.append(assistant_message)
            final_result["content"] = result.get("content", "")
            final_result["tool_calls"].extend(result.get("tool_calls", []))

            # Check stop reason
            finish_reason = result.get("finish_reason", "unknown")
            logger.debug(f"LLM response finish reason: {finish_reason}")

            # Handle tool calls if present
            if result.get("tool_calls") and finish_reason == "tool_calls":
                executed_calls = self._execute_tool_calls(result["tool_calls"])
                final_result["tool_calls"] = executed_calls
                continue  # Continue loop to process tool results
            
            elif finish_reason in ["stop", "length", "content_filter"]:
                break  # Stop if LLM indicates completion
            else:
                logger.warning(f"Unknown finish reason: {finish_reason}")
                break  # Break on unknown finish reason

        # Limit conversation history
        if len(self.conversation_history) > 50:
            self.conversation_history = [self.conversation_history[0]] + self.conversation_history[-49:]

        return final_result

    def _execute_tool_calls(self, tool_calls: List[Dict]) -> List[Dict]:
        """Execute tool calls and append results to conversation history."""
        from agentic.tools.display import ToolExecutionDisplay
        display = ToolExecutionDisplay()
        executed_calls = []

        for tool_call in tool_calls:
            function_name = tool_call["function"]["name"]
            tool_call_id = tool_call.get("id")
            try:
                arguments = json.loads(tool_call["function"]["arguments"])
            except json.JSONDecodeError as e:
                logger.error(f"Invalid arguments for {function_name}: {str(e)}")
                display.show_tool_error(f"Error in {function_name}", str(e))
                tool_call["error"] = str(e)
                executed_calls.append(tool_call)
                continue

            display.show_tool_start(function_name, trusted=True, args=arguments)

            try:
                if function_name in self.tools_registry:
                    result = self.tools_registry[function_name](**arguments)
                else:
                    result = self.tool_manager.execute_tool(function_name, arguments)
                tool_call["result"] = result
                executed_calls.append(tool_call)

                # Append tool result to conversation history
                self.conversation_history.append(Message(
                    role="tool",
                    content=str(result),
                    tool_call_id=tool_call_id
                ))
            except Exception as e:
                logger.error(f"Error executing {function_name}: {str(e)}")
                display.show_tool_error(f"Error in {function_name}", str(e))
                tool_call["error"] = str(e)
                executed_calls.append(tool_call)

        return executed_calls

    def _get_available_tools(self) -> List[Dict]:
        """Get OpenAI-formatted tools for the configured tool names."""
        return self.tool_manager.get_tools(self.config.tools)

    def _format_messages_for_llm(self) -> List[Dict]:
        """Convert Message objects to a format suitable for the LLM client."""
        messages = []
        for msg in self.conversation_history:
            message_dict = {"role": msg.role, "content": msg.content}
            if msg.tool_calls:
                message_dict["tool_calls"] = msg.tool_calls
            if msg.tool_call_id:
                message_dict["tool_call_id"] = msg.tool_call_id
            messages.append(message_dict)
        return messages

    def clear_history(self) -> None:
        """Clear conversation history except system message."""
        self.conversation_history = [Message(role="system", content=self.system_prompt)]

In [3]:
# Example Usage 
config = AgentConfig(
    name="demo_agent",
    instructions="You are a helpful assistant.",
    tools=["fs_write","execute_bash"]
)
llm_client = LLMClient(model="qwen3:14b",base_url="http://192.168.29.147:11500/v1",api_key="ollama")
agent = Agent(config,llm_client)

2025-09-24 01:27:04,719 - INFO - Retrying request to /chat/completions in 0.415726 seconds
2025-09-24 01:27:10,141 - INFO - Retrying request to /chat/completions in 0.975507 seconds
2025-09-24 01:27:15,870 - INFO - HTTP Request: POST http://192.168.29.147:11500/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-24 01:27:15,874 - INFO - Initialized LLM client with model: qwen3:14b


In [4]:
# Test the agent
response = agent.run("get all the nodes present in the k8s cluster")
print(f"Response: {response.get('content', 'No content')}")
print()

2025-09-24 01:27:16,089 - INFO - HTTP Request: POST http://192.168.29.147:11500/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, the user wants to get all the nodes present in the Kubernetes cluster. Let me think about how to approach this.

First, I know that in Kubernetes, nodes are managed by the API server. To list all nodes, the standard command is kubectl get nodes. That should retrieve the information. But I need to make sure that the user has the necessary permissions and that kubectl is configured correctly. However, the user didn't mention any issues with that, so I can assume it's a straightforward case.

Now, looking at the tools available. The user provided two functions: fs_write and execute_bash. The execute_bash function can run bash commands. Since kubectl is a command-line tool, using execute_bash to run "kubectl get nodes" makes sense here.

I need to structure the tool call correctly. The execute_bash function requires the "command" parameter. The command would be "kubectl get nodes". The other parameters like capture_outp

2025-09-24 01:27:26,725 - INFO - HTTP Request: POST http://192.168.29.147:11500/v1/chat/completions "HTTP/1.1 200 OK"


🛠️ Executing execute_bash (trusted)
Command: kubectl get nodes
Summary : List all Kubernetes nodes

╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, the user asked to get all the nodes present in the Kubernetes cluster. I used the execute_bash function to run the command "kubectl get nodes". The output shows three nodes: control-plane, workera, and workerb. All of them are in the Ready status. The control-plane has the control-plane role, while the other two are worker nodes. The versions are all v1.31.8, and they've been around for about 149 days. I should present this information clearly, listing each node's name, status, roles, age, and version. Make sure to mention that the control-plane is a control node and the others are workers. Also, note the age and version一致性. Keep it concise but informative.
 │
╰────────────────────────────────────────────────────────────╯


The Kubernetes cluster has the following nodes:

...

 • Roles: None (worker node)

 • Age: 149 days

 • Version: v1.31.8

All nodes are running the same Kubernetes version and have been active for approximately 149 days. The cluster     
appears to be stable with one control plane node and two worker nodes.

Response: 

The Kubernetes cluster has the following nodes:

1. **control-plane**
   - Status: Ready
   - Roles: control-plane
   - Age: 149 days
   - Version: v1.31.8

2. **workera**
   - Status: Ready
   - Roles: None (worker node)
   - Age: 149 days
   - Version: v1.31.8

3. **workerb**
   - Status: Ready
   - Roles: None (worker node)
   - Age: 149 days
   - Version: v1.31.8

All nodes are running the same Kubernetes version and have been active for approximately 149 days. The cluster appears to be stable with one control plane node and two worker nodes.

